In [186]:
# !pip install pypdf3
# !pip install tabula-py
# !pip install camelot-py

import pandas as pd
import numpy as np
# import camelot as camel
from PyPDF3 import PdfFileReader 
from tabula import read_pdf 
from tabulate import tabulate

from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2


# https://www.youtube.com/watch?v=702lkQbZx50 - for tabula
# https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates - for tabula 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
import os 
import re

list_of_files = os.listdir('../data/input/berlin_adresses/2023')
list_of_files

['adr_charlottenburg-wilmdersdorf_2023.pdf',
 'adr_friedrichshain-kreuzberg_2023.pdf',
 'adr_lichtenberg_2023.pdf',
 'adr_marzahn-hellersdorf_2023.pdf',
 'adr_mitte_2023.pdf',
 'adr_neukoelln_2023.pdf',
 'adr_pankow_2023.pdf',
 'adr_reinickendorf_2023.pdf',
 'adr_spandau_2023.pdf',
 'adr_steglitz-zehlendorf_2023.pdf',
 'adr_tempelhof-schoeneberg_2023.pdf',
 'adr_treptow-koepenick_2023.pdf']

In [188]:
    # import_data = read_pdf('../data/input/berlin_adresses/2023/adr_charlottenburg-wilmdersdorf_2023.pdf', pages = 30, encoding = 'ISO-8859-1', stream = True, area = [164, 33, 783, 520], guess = True, pandas_options={'header': None})
    # table_df = import_data[0]
    # # table_df. pop_data.insert(0, 'year', year)
    # table_df


In [189]:
    # import_data = read_pdf('../data/input/berlin_adresses/2023/adr_charlottenburg-wilmdersdorf_2023.pdf', pages = 54, encoding = 'ISO-8859-1', stream = True, area = [164, 33, 783, 520], guess = True, pandas_options={'header': None})
    # table_df = import_data[0]
    # columns_len = len(table_df.columns)
    # print(columns_len)
    
    # display(table_df.head(2))
    
    # if columns_len < 14:
    #     if table_df.iloc[:, 3].dtype in ['int64', 'int32', 'float64', 'float32']:
    #         table_df.insert(3, 'm1', np.nan)

    #     if table_df.iloc[:, 6].dtype in ['int64', 'int32', 'float64', 'float32']:
    #         table_df.insert(5, 'm2', np.nan)
        
        
    #     column_length = len(table_df.columns)

        
    #     if column_length < 14:
    #        table_df['m3'] = np.nan
            
        
        
    # table_df

In [198]:
def extract_all_pages_from_pdf(path, file_name):
    # open path as binary / rb = read binary -> used for pdf files (https://www.youtube.com/watch?v=OZo2HxoIOtw)
            
    pdf = PdfFileReader(open(path, 'rb'))
    last_page = pdf.getNumPages()
    last_page += 1

    # defining page range for import 
    [i for i in range(0, 15)]
    page_range = [i for i in range(5, last_page)]
    print(page_range)

    # prepare column name filler
    column_names = [i for i in range(0, 14)]
    print(colum_names)

    # create empty dataframe
    adress_data = pd.DataFrame(columns = column_names)
    display(adress_data)
    
    # extract district information
    district = [re.findall(r'(?<=adr_)(.*?)(?=_2023\.pdf)', file_name)]
    print(district)

    for page in page_range:
        
        import_data = read_pdf(path, pages = page, encoding = 'ISO-8859-1', stream = True, area = [175, 33, 783, 520], guess = False, pandas_options={'header': None})
        table_df = import_data[0]

        
        table_df = import_data[0]
        columns_len = len(table_df.columns)

        if columns_len < 14:
            if table_df.iloc[:, 3].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(3, 'm1', np.nan)

            if table_df.iloc[:, 6].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(5, 'm2', np.nan)
            
            
            column_length = len(table_df.columns)

            
            if column_length < 14:
                table_df['m3'] = np.nan
            
            
            table_df.columns = column_names
            table_df['14'] = str(district[0][0])
            
            adress_data = pd.concat([adress_data, table_df], ignore_index=True , axis=0)
            
            print(f'page: {page}')
            print(adress_data.shape)
    
    return adress_data, district[0][0]




In [199]:


for file_name in list_of_files:
    path = f'../data/input/berlin_adresses/2023/{file_name}'
    print(path)
    
    dataset, name = extract_all_pages_from_pdf(path, file_name)
    
    dataset.to_csv(f'../data/output/temp_adress_data{name}-2023.csv', index = False)
    display(dataset.head(5))
    print(dataset.shape)
    
# # open path as binary / rb = read binary -> used for pdf files (https://www.youtube.com/watch?v=OZo2HxoIOtw)
        
# pdf = PdfFileReader(open(path, 'rb'))
# last_page = pdf.getNumPages()
# last_page += 1


../data/input/berlin_adresses/2023/adr_charlottenburg-wilmdersdorf_2023.pdf
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13


[['charlottenburg-wilmdersdorf']]
page: 5
(48, 15)
page: 6
(96, 15)
page: 7
(144, 15)
page: 8
(192, 15)
page: 9
(240, 15)
page: 10
(288, 15)
page: 11
(336, 15)
page: 13
(384, 15)
page: 14
(432, 15)
page: 15
(480, 15)
page: 16
(528, 15)
page: 17
(576, 15)
page: 19
(624, 15)
page: 20
(672, 15)
page: 22
(720, 15)
page: 23
(768, 15)
page: 24
(816, 15)
page: 25
(864, 15)
page: 26
(912, 15)
page: 27
(960, 15)
page: 28
(1008, 15)
page: 29
(1056, 15)
page: 32
(1104, 15)
page: 33
(1152, 15)
page: 34
(1200, 15)
page: 37
(1248, 15)
page: 38
(1296, 15)
page: 42
(1344, 15)
page: 43
(1392, 15)
page: 44
(1440, 15)
page: 45
(1488, 15)
page: 49
(1536, 15)
page: 50
(1584, 15)
page: 51
(1632, 15)
page: 52
(1680, 15)
page: 54
(1701, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,Aachener Straße,1,NaN,45,NaN,U,4,50,11,48,Wilmd,10713,NaN,charlottenburg-wilmdersdorf
1,1,Aachener Straße,2,NaN,44,NaN,G,4,50,11,48,Wilmd,10713,NaN,charlottenburg-wilmdersdorf
2,5,Abbestraße,1,NaN,19,NaN,U,4,30,6,22,Charl,10587,NaN,charlottenburg-wilmdersdorf
3,5,Abbestraße,2,NaN,12,NaN,G,4,30,6,22,Charl,10587,NaN,charlottenburg-wilmdersdorf
4,8793,Adam-von-Trott-Straße,1,NaN,7,NaN,U,4,10,1,1,CharN,13627,NaN,charlottenburg-wilmdersdorf


(1701, 15)
../data/input/berlin_adresses/2023/adr_friedrichshain-kreuzberg_2023.pdf
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


...

(1526, 15)
../data/input/berlin_adresses/2023/adr_treptow-koepenick_2023.pdf
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13


[['treptow-koepenick']]
page: 5
(48, 15)
page: 6
(96, 15)
page: 8
(144, 15)
page: 9
(192, 15)
page: 10
(240, 15)
page: 11
(288, 15)
page: 12
(336, 15)
page: 14
(384, 15)
page: 17
(432, 15)
page: 19
(480, 15)
page: 20
(528, 15)
page: 21
(576, 15)
page: 22
(624, 15)
page: 23
(672, 15)
page: 24
(720, 15)
page: 25
(768, 15)
page: 26
(816, 15)
page: 28
(864, 15)
page: 29
(912, 15)
page: 31
(960, 15)
page: 32
(1008, 15)
page: 33
(1056, 15)
page: 35
(1104, 15)
page: 36
(1152, 15)
page: 37
(1200, 15)
page: 38
(1248, 15)
page: 39
(1296, 15)
page: 40
(1344, 15)
page: 41
(1392, 15)
page: 42
(1440, 15)
page: 43
(1488, 15)
page: 44
(1536, 15)
page: 45
(1584, 15)
page: 46
(1632, 15)
page: 48
(1680, 15)
page: 49
(1728, 15)
page: 50
(1776, 15)
page: 51
(1824, 15)
page: 52
(1872, 15)
page: 53
(1920, 15)
page: 55
(1968, 15)
page: 56
(2016, 15)
page: 57
(2064, 15)
page: 58
(2112, 15)
page: 59
(2160, 15)
page: 60
(2208, 15)
page: 61
(2256, 15)
page: 62
(2282, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,9497,Aalstieg,1,NaN,17,NaN,U,9,50,18,38,Rahnd,12589,NaN,treptow-koepenick
1,9497,Aalstieg,2,NaN,38,NaN,G,9,50,18,38,Rahnd,12589,NaN,treptow-koepenick
2,11206,Abendseglersteig,2,NaN,78,NaN,G,9,50,18,38,Rahnd,12589,NaN,treptow-koepenick
3,11206,Abendseglersteig,11,NaN,75,NaN,U,9,50,18,38,Rahnd,12589,NaN,treptow-koepenick
4,8834,Abram-Joffe-Straße,2,NaN,18,NaN,G,9,20,7,15,Adler,12489,NaN,treptow-koepenick


(2282, 15)


In [ ]:
# import_data = read_pdf('../data/input/berlin_adresses/2023/adr_charlottenburg-wilmdersdorf_2023.pdf', pages = '5', encoding = 'ISO-8859-1', stream = True, area = [175, 33, 783, 520], guess = False, pandas_options={'header': None})

# # was saved as a list -> need to transformt it into dataframe 
# table_df = import_data[0]

# table_df